In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"
from math import ceil

import time
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import tensorflow as tf
from tensorflow.python.training.summary_io import SummaryWriterCache

import graph_manager
import net
import loss_utils
import tf_inputs
import tf_utils
import viz

import logging
logging.getLogger("tensorflow").setLevel(logging.WARNING)

## Base Configuration

--- 

In [2]:
data = 'vedai'
data = 'stanford'
#data = 'dota'

configuration = {}
if data == 'vedai':
    configuration['setting'] = 'vedai_merged_1s'
    configuration['image_folder'] = '/home/aroyer/Datasets/VEDAI/Vehicules1024/'
    configuration['exp_name'] = 'aerial_nips/vedai'
    configuration['retrieval_top_n'] = 100
elif data == 'stanford':
    configuration['setting'] = 'stanford_1s'
    configuration['image_folder'] = '/home/aroyer/Datasets/stanford_images'
    configuration['exp_name'] = 'aerial_nips/stanford'
    configuration['retrieval_top_n'] = 150
elif data == 'dota':
    configuration['setting'] = 'dota_merged_1s'
    configuration['image_folder'] = '/fs3/group/chlgrp/datasets/DOTA/%s/images'
    configuration['image_folder'] = '/home/aroyer/Datasets/DOTA/%s/images'
    configuration['exp_name'] = 'aerial_nips/dota'
    configuration['retrieval_top_n'] = 700

    
## Metadata
configuration['save_summaries_steps'] = 1
tfrecords_path = '/home/aroyer/indolentDetect/TFRecords/metadata_%s.txt'
metadata = graph_manager.load_metadata(tfrecords_path % configuration['setting'])
configuration.update(metadata)
configuration['num_classes'] = len(configuration['data_classes'])
configuration['feature_keys'] = [x for x in configuration['feature_keys'] 
                                 if x != 'clustered_bounding_boxes' and x != 'num_clustered_boxes']

## GPUs
configuration['num_gpus'] = 2                               
configuration['gpu_mem_frac'] = 1

## Inputs Pipeline
configuration['subset'] = -1
configuration['image_size'] = 256
configuration['batch_size'] = 16
configuration['val_batch_size'] = 16
configuration['shuffle_buffer'] = 1000
    
## Evaluation
configuration['save_checkpoint_secs'] = 3600
configuration['retrieval_top_n'] = 100
configuration['retrieval_intersection_threshold'] = [0.25, 0.5, 0.75]

## Training
configuration['learning_rate'] = 1e-3
configuration['centers_localization_loss_weight'] = 1.
configuration['scales_localization_loss_weight']  = 1.
configuration['confidence_loss_weight']  = 5.
configuration['noobj_confidence_loss_weight']  = 1.
configuration['classification_loss_weight']  = 1.
configuration['offsets_loss_weight']  = 1.
configuration['target_conf_fn'] = 'iou'
configuration['assignment_reward_fn'] = 'iou'

graph_manager.finalize_configuration(configuration)

10476 training samples (328 iters)
2619 validation samples (82 iters)
grid size [8 8]

Config:
assignment_reward_fn: iou
batch_size: 16
centers_localization_loss_weight: 1.0
classification_loss_weight: 1.0
confidence_loss_weight: 5.0
data_classes: ['Biker', 'Bus', 'Car', 'Cart', 'Pedestrian', 'Skater']
exp_name: aerial_nips/stanford
feature_keys: ['im_id', 'num_boxes', 'bounding_boxes', 'classes']
gpu_mem_frac: 1
image_folder: /home/aroyer/Datasets/stanford_images
image_size: 256
last_val_batch_size: 27
learning_rate: 0.001
multistage: False
noobj_confidence_loss_weight: 1.0
num_cells: [8 8]
num_classes: 6
num_gpus: 2
offsets_loss_weight: 1.0
retrieval_intersection_threshold: [0.25, 0.5, 0.75]
retrieval_top_n: 100
save_checkpoint_secs: 3600
save_summaries_steps: 1
scales_localization_loss_weight: 1.0
setting: stanford_1s
shuffle_buffer: 1000
subset: -1
target_conf_fn: iou
train_max_num_bbs: 107
train_num_iters_per_epoch: 328
train_num_samples: 10476
train_num_samples_per_iter: 32
train

In [3]:
from importlib import reload
reload(tf_utils)
reload(graph_manager)
reload(loss_utils)
reload(net)
reload(tf_inputs)
reload(viz);

## YOLOv2
---

### loss function (loss_utils.get_yolo_v2_loss)

  * `num_cells` (**None**): A 2D int tensor indicating the number of cells on each axis
  * `base_name` ('net'): Prefix for the losses tensors names.
  * `gt_bbs_key` ('bounding_boxes) Key to ground-truth bounding boxes in  inputs
  * `obj_i_mask_key` ('obj_i_mask_bbs') Key to the ground-truth obj_i_mask in inputs
  * `centers_localization_loss_weight` (1.) Weights for the localization losses of centers
  * `scales_localization_loss_weight` (1.) Weights for the localization losses of log scales
  * `confidence_loss_weight` (5.) Weights for the confidence loss
  * `noobj_confidence_loss_weight` (1.) Weights for the confidence loss  (empty cells)
  * `classification_loss_weight` (1.) Weights for the counting loss
  * `offsets_loss_weight` (1.) Weights for the offsets loss
  * `target_conf_fn` ('iou') Function to compute target confidences. See `tf_utils.get_*`
  * `assignment_reward_fn` ('iou') Function to compute assignment reward. See `tf_utils.get_*`

### evaluation function (loss_utils.get_yolo_v2_eval)

  * `num_cells` (**None**): A 2D int tensor indicating the number of cells on each axis
  * `base_name` ('net'): Prefix for the losses tensors names.
  * `gt_bbs_key` ('bounding_boxes) Key to ground-truth bounding boxes in  inputs
  * `gt_num_bbs_key` ('num_boxes') Key to the ground-truth number of boxes in inputs
  * `retrieval_top_n` (100) Max number of bbs to retrieve
  * `retrieval_intersection_threshold` ([0.25, 0.5, 0.75]) Compute map (resp. recall) at the given iou (resp. inter) threshold
  * `retrieval_confidence_threshold` (0.1) Confidence threshold for predictions
  * `with_nms` (False) Whether to apply non maximum suppression
  * `nms_threshold` (0.5) IOU threshold for non maximum suppression

In [4]:
def forward_pass(inputs, 
                 outputs, 
                 configuration,
                 reuse=False, 
                 verbose=False,
                 scope_name='model'):
    """Forward-pass in the net"""
    with tf.variable_scope(scope_name, reuse=reuse):
        activations = net.tiny_yolo_v2(
            inputs["image"], is_training=False, reuse=reuse, verbose=verbose, **configuration)
        net.get_outputs_yolov2(activations, outputs, reuse=reuse,  verbose=verbose,  **configuration)
            
    
def eval_pass(inputs, metrics_to_norms, clear_metrics_op, update_metrics_op, configuration, 
              device=0, is_chief=False):
    """ Compute output of the net and add metrics update and reset operatiosn to the graph"""
    outputs = {}
    base_name = graph_manager.get_defaults(configuration, ['base_name'], verbose=is_chief)[0]
    if is_chief: print(' \033[34m%s:\033[0m' % base_name)
        
    with tf.name_scope('%s/net' % base_name):
        forward_pass(inputs, outputs, configuration, scope_name=base_name, 
                     reuse=not is_chief, verbose=is_chief) 
        
    with tf.name_scope('%s/eval' % base_name):
        graph_manager.add_metrics_to_graph(
            loss_utils.get_yolo_v2_eval, inputs, outputs, metrics_to_norms, clear_metrics_op, 
            update_metrics_op, configuration, device=device, verbose=is_chief)      
    return outputs    

## Multistage YOLO

---

### Extract groups / patches

  * `confidence_threshold`: only keep output boxes with the given confidence
  * `nms_threshold`: Apply non maximum suppression with the following threshold
  * `num_crops`: Number of crops to extract per image

In [5]:
def extract_groups(inputs, 
                   outputs,
                   verbose=False,
                   epsilon=1e-8,
                   **kwargs):    
    (confidence_threshold, nms_threshold, num_outputs) = graph_manager.get_defaults(
        kwargs, ['val_patch_confidence_threshold', 'patch_nms_threshold', 'val_num_crops'], 
        verbose=verbose)
    if verbose:        
        print('    extracting \x1b[32m%d\x1b[0m crops' % num_outputs)
        
    ## Flatten
    # predicted_score: (batch, num_boxes)
    # predicted_boxes: (batch, num_boxes, 4)
    # predicted_offsets: (batch, num_boxes, 1)
    with tf.name_scope('flat_output'):
        predicted_scores = tf_utils.flatten_percell_output(outputs["detection_scores"])
        predicted_boxes = tf_utils.flatten_percell_output(outputs["bounding_boxes"])
        
    # During test time we only extract crops if groups or low confidence individuals
    with tf.name_scope('filter_groups'):
        if 'group_classification_logits' in outputs:
            strong_confidence_threshold = graph_manager.get_defaults(
                kwargs, ['val_patch_strong_confidence_threshold'], verbose=verbose)[0]
            # is_group: (batch, num_boxes, 1)
            is_group = tf.to_float(tf.nn.sigmoid(outputs['group_classification_logits']) > 0.5)
            num_boxes = outputs['bounding_boxes'].get_shape()[-2].value
            is_group = tf.expand_dims(is_group, axis=-2)
            is_group = tf.tile(is_group, (1, 1, 1, num_boxes, 1))
            is_group = tf_utils.flatten_percell_output(is_group)
            # should_be_refined: (batch, num_boxes, 1)
            is_not_strongly_confident = tf.to_float(predicted_scores <= strong_confidence_threshold)
            should_be_refined = tf.minimum(1., is_group + is_not_strongly_confident)
            # Add confident single boxes as additional output of (batch, num_boxes, 4) shape
            outputs['added_detection_scores'] = predicted_scores * (1. - should_be_refined)
            outputs['added_bounding_boxes'] = predicted_boxes * (1. - should_be_refined)
            # Filter them out from potential crops
            predicted_scores *= should_be_refined
            predicted_boxes *= should_be_refined
            
        # Ignore class information to sort boxes for crops
        predicted_scores = tf.reduce_max(predicted_scores, axis=-1)
        
    ## Filter out low confidences
    with tf.name_scope('filter_confidence'):
        filtered = tf.to_float(predicted_scores > confidence_threshold)
        predicted_scores *= filtered
        predicted_boxes *= tf.expand_dims(filtered, axis=-1)
        
    ## Rescale boxes if learned offsets
    with tf.name_scope('offsets_rescale_boxes'):
        if 'offsets' in outputs:
            predicted_offsets = tf_utils.flatten_percell_output(outputs["offsets"])
            predicted_boxes = tf_utils.rescale_with_offsets(predicted_boxes, predicted_offsets, epsilon)
    
    ## Non-maximum suppression
    with tf.name_scope('nms'):
        nms_boxes = []
        nms_boxes_confidences = []
        for i in range(inputs['batch_size']):
            boxes, scores = tf_utils.nms_with_pad(predicted_boxes[i, :, :], 
                                                  predicted_scores[i, :],
                                                  num_outputs, 
                                                  iou_threshold=nms_threshold)
            nms_boxes.append(boxes)
            nms_boxes_confidences.append(scores)
        nms_boxes = tf.stack(nms_boxes, axis=0) 
        nms_boxes = tf.reshape(nms_boxes, (-1, num_outputs, 4))
        nms_boxes_confidences = tf.stack(nms_boxes_confidences, axis=0) 
        nms_boxes_confidences = tf.reshape(nms_boxes_confidences, (-1, num_outputs))
        
    ## Return
    outputs['crop_boxes'] = nms_boxes
    outputs['crop_boxes_confidences'] = nms_boxes_confidences
    return nms_boxes, nms_boxes_confidences

### Graph utils

In [6]:
def feed_pass(inputs, outputs, configuration, is_chief=False):
    """
        Args:
            inputs: inputs dictionnary
            outputs: outputs dictionnary
            configuration: config dictionnary
        
        Returns:
            Dictionnary of inputs for the next stage
    """
    if is_chief: print(' \033[34mextract patches:\033[0m')
    with tf.variable_scope('extract_patches'):
        crop_boxes, crop_boxes_confidences = extract_groups(
            inputs, outputs, verbose=is_chief, **configuration)
    return graph_manager.get_stage2_inputs(inputs, crop_boxes, mode='val', verbose=is_chief, **configuration)
            
                
def eval_pass_withmerge(stage2_inputs, stage1_inputs, stage1_outputs, configuration, metrics_to_norms, 
                        clear_metrics_op, update_metrics_op, device=0, is_chief=False):
    """ Evaluation for the second stage: We rescale the bounding boxes after the first stage and compare to the
    original ground-truth
        Args:
            stage2_inputs: inputs dictionnary for stage2
            stage1_inputs: inputs dictionnary for stage 1
            stage1_outputs: outputs dictionnary for stage1
            configuration: config dictionnary
            metrics_to_norms: Map metrics key to normalizer key
            clear_metrics_op: List to be updated with reset operations
            update_metrics_op: List to be updated with update operation
            device: Current device number to be used in the variable scope for each metric
        
        Returns:
            Dictionnary of outputs, merge by image
            Dictionnary of unscaled ouputs (for summary purposes)
    """
    ## Feed-forward pass
    base_name = graph_manager.get_defaults(configuration, ['base_name'], verbose=is_chief)[0]
    if is_chief: print(' \033[34m%s:\033[0m' % base_name)
    outputs = {}
    with tf.name_scope('net'):
        forward_pass(stage2_inputs, outputs, configuration, scope_name=base_name,
                     reuse=not is_chief, verbose=is_chief) 
            
    ## Add the number of useful crops in metrics
    # crop_boxes/confidences : (batch_size, num_crops, (4))
    with tf.name_scope('num_valid_crops'):
        crop_boxes = stage1_outputs["crop_boxes"]
        num_crop_boxes = tf.reduce_sum(tf.to_float(tf.logical_and(crop_boxes[..., 2] > crop_boxes[..., 0],
                                                                  crop_boxes[..., 3] > crop_boxes[..., 1])))
        with tf.variable_scope('metrics_dev_%d' % device):
            metric = tf.get_variable('num_crops_running', shape=(), initializer=tf.zeros_initializer(), 
                                     trainable=False, collections=[tf.GraphKeys.GLOBAL_VARIABLES, 'num_crops'])
        metrics_to_norms['num_crops'] = '%s_num_samples_eval' % base_name
        clear_metrics_op.append(tf.assign(metric, tf.constant(0., dtype=tf.float32, shape=())))
        update_metrics_op.append(tf.assign_add(metric, num_crop_boxes))    
    
    ## Rescale outputs of each crop relatively to their original image
    unscaled_outputs = {key: outputs[key] for key in ['bounding_boxes', 'detection_scores']} # for summary
    with tf.variable_scope('rescale_crops'):
        # Rescale confidence scores
        with tf.name_scope('conditioned_detection_scores'):
            crop_boxes_confidences = stage1_outputs["crop_boxes_confidences"]
            outputs['detection_scores'] *= tf.reshape(crop_boxes_confidences, (-1, 1, 1, 1, 1))
            
        # Reshape outputs from stage2 to stage1
        with tf.name_scope('reshape_stage2_outputs'):
            num_crops = crop_boxes.get_shape()[1].value
            num_boxes = outputs['bounding_boxes'].get_shape()[-2].value
            batch_size = graph_manager.get_defaults(configuration, ['val_batch_size'], verbose=is_chief)[0]
            # outputs:  (stage1_batch * num_crops, num_cell, num_cell, num_boxes, ...)
            # to: (stage1_batch, num_cell, num_cell, num_boxes * num_crops, ...)
            for key, value in outputs.items():
                shape = tf.shape(value)
                batches = tf.split(value, batch_size, axis=0)
                batches = [tf.concat(tf.unstack(b, num=num_crops, axis=0), axis=2) for b in batches]
                outputs[key] = tf.stack(batches, axis=0)
    
        # Rescale bounding boxes from stage2 to stage1
        with tf.name_scope('rescale_stage2_outputs'):
            # crop_boxes: (stage1_batch, 1, 1, num_crops * num_boxes, 4)
            crop_boxes = tf.reshape(crop_boxes, (batch_size, num_crops, 1, 4))
            crop_boxes = tf.tile(crop_boxes, (1, 1, num_boxes, 1))
            crop_boxes = tf.reshape(crop_boxes, (batch_size, 1, 1, num_crops * num_boxes, 4))
            crop_mins, crop_maxs = tf.split(crop_boxes, 2, axis=-1)
            # bounding_boxes: (stage1_batch, num_cells, num_cells, num_crops * num_boxes, 4)
            bounding_boxes = outputs['bounding_boxes']
            bounding_boxes *= tf.maximum(1e-8, tf.tile(crop_maxs - crop_mins, (1, 1, 1, 1, 2)))
            bounding_boxes += tf.tile(crop_mins, (1, 1, 1, 1, 2))
            bounding_boxes = tf.clip_by_value(bounding_boxes, 0., 1.)
            outputs['bounding_boxes'] = bounding_boxes
            
    # Additional bounding boxes outputs propagated from stage 1
    if 'added_detection_scores' in stage1_outputs:
        outputs['added_detection_scores'] = stage1_outputs['added_detection_scores']
        del stage1_outputs['added_detection_scores']
    if 'added_bounding_boxes' in stage1_outputs:
        outputs['added_bounding_boxes'] = stage1_outputs['added_bounding_boxes']
        del stage1_outputs['added_bounding_boxes']
        
    # Evaluate `output` versus the initial (stage 1) inputs
    with tf.name_scope('eval'):
        graph_manager.add_metrics_to_graph(
            loss_utils.get_yolo_v2_eval, stage1_inputs, outputs, metrics_to_norms, clear_metrics_op, 
            update_metrics_op, configuration, device=device, verbose=is_chief)
    return outputs, unscaled_outputs

## Train multistage YOLO
--- 

In [9]:

multistage_configuration = configuration.copy()
multistage_configuration['exp_name'] += '/finetune_multistage'
multistage_configuration['full_image_size'] = 1024
multistage_configuration['summary_confidence_thresholds'] = [0.25, 0.5]
graph_manager.finalize_grid_offsets(multistage_configuration)
stage1_configuration = multistage_configuration.copy()
stage2_configuration = multistage_configuration.copy()

## Choices
# setting 1
multistage_configuration['model_path'] = 'log/aerial_nips/stanford/multistage/05-05_15-33/'
stage1_configuration['image_size'] = 256
with_groups = True
stage2_configuration['val_num_crops'] = 10

# setting 2
multistage_configuration['model_path'] = 'log/aerial_nips/stanford/multistage/05-05_22-07/'
stage1_configuration['image_size'] = 512
with_groups = True
stage2_configuration['val_num_crops'] = 10

## Finalize
multistage_configuration['model_path'] = tf.train.latest_checkpoint(multistage_configuration['model_path'])
stage1_configuration['num_boxes'] = 1
if data == 'vedai':
    stage2_configuration['num_boxes'] = 1
elif data == 'stanford':
    stage2_configuration['num_boxes'] = 1 if stage1_configuration['image_size'] >= 256 else 5
elif data == 'dota':
    stage2_configuration['num_boxes'] = 1 if stage1_configuration['image_size'] >= 256 else 5

if with_groups:
    stage1_configuration['with_groups'] = True
    stage1_configuration['with_group_classification'] = True
    stage1_configuration['gt_bbs_key'] = 'clustered_bounding_boxes'
    stage1_configuration['gt_num_bbs_key'] = 'num_clustered_boxes'
    stage1_configuration['obj_i_mask_key'] = 'obj_i_mask_clustered_bbs'
else:
    stage1_configuration['with_groups'] = False
    stage1_configuration['gt_bbs_key'] = 'bounding_boxes'
    stage1_configuration['gt_num_bbs_key'] = 'num_boxes'
    stage1_configuration['obj_i_mask_key'] = 'obj_i_mask_bbs'

stage1_configuration.update({
    'assignment_reward_fn': 'iou',
    'target_conf_fn': 'iou',
    'with_offsets': True,
    'base_name': 'stage1'
})
graph_manager.finalize_grid_offsets(stage1_configuration)

stage2_configuration.update({
    'gt_bbs_key': 'bounding_boxes',
    'gt_num_bbs_key': 'num_boxes',
    'obj_i_mask_key': 'obj_i_mask_bbs',
    'target_conf_fn': 'iou',
    'assignment_reward_fn': 'iou',
    'base_name': 'stage2'
})
stage2_configuration['image_size'] = max(128, stage1_configuration['image_size'] // 2)
graph_manager.finalize_grid_offsets(stage2_configuration)
    

with tf.Graph().as_default() as graph:          
    ########################################################################## Config
    with tf.name_scope('config_summary'):
        viz.add_text_summaries(stage1_configuration, family="stage1") 
        viz.add_text_summaries(stage2_configuration, family="stage2") 
        config_summary_op = tf.summary.merge_all(key='config')
    
    
    ##########################################################################  Evaluation graph
    with tf.name_scope('eval'):        
        print('\n\033[43mVal Graph:\033[0m')
        update_metrics_op = []    # Store operations to update the metrics
        clear_metrics_op = []     # Store operations to reset the metrics
        metrics_to_norms = {}
        inputs = graph_manager.get_inputs(mode='val', verbose=False, **stage1_configuration)         
        viz.display_graph_size('inputs(val)')            
        global_step = tf.train.create_global_step()
        
        for i, val_inputs in enumerate(inputs):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('dev%d' % i):
                    is_chief = (i == 0)
                    val_s1_outputs = eval_pass(val_inputs, metrics_to_norms, 
                                               clear_metrics_op, update_metrics_op, 
                                               stage1_configuration, device=i, is_chief=is_chief) 
                    val_s2_inputs = feed_pass(val_inputs, val_s1_outputs, stage2_configuration,
                                            is_chief=is_chief)
                    val_s2_outputs, val_s2_unscaled_outputs = eval_pass_withmerge(
                        val_s2_inputs, val_inputs,  val_s1_outputs, stage2_configuration, 
                        metrics_to_norms, clear_metrics_op, update_metrics_op, device=i, is_chief=is_chief)
                    if is_chief:
                        with tf.name_scope('stage1'):
                            graph_manager.add_summaries(val_inputs, val_s1_outputs, mode='val', 
                                                        **stage1_configuration)   
                        with tf.name_scope('stage2'):
                            graph_manager.add_summaries(val_inputs, val_s2_outputs, mode='val', verbose=0,
                                                        display_inputs=0, **stage2_configuration)
                            graph_manager.add_summaries(val_s2_inputs, val_s2_unscaled_outputs, mode='val',
                                                        verbose=0, **stage2_configuration) 
            viz.display_graph_size('val net (gpu:%d)' % i)

        with tf.name_scope('eval'):
            print('    \x1b[32m%d\x1b[0m eval update ops' % len(update_metrics_op))
            print('    \x1b[32m%d\x1b[0m eval clear ops' % len(clear_metrics_op))
            update_metrics_op = tf.group(*update_metrics_op)
            clear_metrics_op = tf.group(*clear_metrics_op)
            eval_summary_op, eval_values_op = graph_manager.get_eval_op(metrics_to_norms, True)
        
        # Additional info
        print('\n\033[43mEval metrics:\033[0m')
        print('\n'.join(["    \033[35m%s:\033[0m %s tensors" % (x, len(tf.get_collection(x)))  
                        for x in tf.get_default_graph().get_all_collection_keys() 
                        if x.endswith('_eval')]))
        
    ########################################################################## Run    
    try:
        print('\n\033[44mLaunch session:\033[0m')
        graph_manager.generate_log_dir(multistage_configuration)
        summary_writer = SummaryWriterCache.get(multistage_configuration["log_dir"])
        eval_dir = os.path.join(multistage_configuration["log_dir"], "eval")
        if not os.path.exists(eval_dir): os.makedirs(eval_dir)
        
        #with graph_manager.get_monitored_training_session(**multistage_configuration) as sess: 
        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=multistage_configuration['gpu_mem_frac']),
            log_device_placement=True,allow_soft_placement=True)
        with tf.Session(config=config) as sess:     
            saver = tf.train.Saver(name='restore_saver')
            saver.restore(sess,  multistage_configuration['model_path'])
            loss_widget = widgets.HTML(value="")
            start_time = time.time()
            global_step_ = 0
            print('\n\033[44mStart evaluating:\033[0m')
            display(loss_widget)   
            num_epochs = multistage_configuration["val_num_iters_per_epoch"]
            sess.run(clear_metrics_op)

            for epoch in range(num_epochs):
                viz.display_eval(loss_widget, global_step_, epoch + 1, num_epochs, start_time)
                sess.run(update_metrics_op) 
                if epoch == num_epochs - 1:
                    eval_summary, eval_values = sess.run([eval_summary_op, eval_values_op])
            config_summary = sess.run(config_summary_op)

            # Write summary
            summary_writer.add_summary(eval_summary, global_step_)
            summary_writer.flush()
            summary_writer.add_summary(config_summary, global_step_)
            summary_writer.flush()
            print('\n'.join('"%s": %s' % (key, value) for key, value in sorted(
                eval_values.items(), key=lambda x: x[0])))
                
    except KeyboardInterrupt:
        print('\nInterrupted at step %d' % global_step_)   


Val Graph:
    pad val inputs with 5 dummy samples
inputs(val) graph: 0.00 MB
 stage1:
  > Use custom tiny yolo v2
    with default `weight_decay` = 0.0
    with default `normalizer_decay` = 0.9
    with default `num_filters` = [16, 32, 64, 128, 256, 512, 1024]
    with default `with_classification` = False
    Output layer shape (?, 16, 16, 1, 7) 
    with default `retrieval_confidence_threshold` = 0.0
    with default `with_nms` = True
    ground-truth key clustered_bounding_boxes
    ground-truth num key num_clustered_boxes
    with default `nms_threshold` = 0.5
 extract patches:
    with default `val_patch_confidence_threshold` = 0.15
    with default `patch_nms_threshold` = 0.25
    extracting 10 crops
    with default `val_patch_strong_confidence_threshold` = 0.75
    with default `patch_intersection_ratio_threshold` = 0.25
    image: shape=[None, 256, 256, 3], dtype=<dtype: 'float32'>
    obj_i_mask_bbs: shape=[None, 8, 8, 1, 107], dtype=<dtype: 'float32'>
    bounding_boxes: s

HTML(value='')

"num_crops": 9.70676
"stage1_map_at0.25_eval": 0.797403
"stage1_map_at0.50_eval": 0.412992
"stage1_map_at0.75_eval": 0.0683188
"stage1_retrieval_rate_at0.25_eval": 0.997068
"stage1_retrieval_rate_at0.50_eval": 0.992909
"stage1_retrieval_rate_at0.75_eval": 0.976534
"stage2_map_at0.25_eval": 0.614529
"stage2_map_at0.50_eval": 0.401338
"stage2_map_at0.75_eval": 0.0573308
"stage2_retrieval_rate_at0.25_eval": 0.728451
"stage2_retrieval_rate_at0.50_eval": 0.681081
"stage2_retrieval_rate_at0.75_eval": 0.495725


In [8]:
#retrieval rate

multistage_configuration['model_path'] = 'log/aerial_nips/stanford/multistage/05-05_15-33/'
stage1_configuration['image_size'] = 256
with_groups = True
stage2_configuration['val_num_crops'] = 10
"num_crops": 8.8194
"stage1_map_at0.25_eval": 0.529613
"stage1_map_at0.50_eval": 0.218881
"stage1_map_at0.75_eval": 0.0387869
"stage1_retrieval_rate_at0.25_eval": 0.97129
"stage1_retrieval_rate_at0.50_eval": 0.928144
"stage1_retrieval_rate_at0.75_eval": 0.837059
"stage2_map_at0.25_eval": 0.433628
"stage2_map_at0.50_eval": 0.243574
"stage2_map_at0.75_eval": 0.0342256
"stage2_retrieval_rate_at0.25_eval": 0.512848
"stage2_retrieval_rate_at0.50_eval": 0.420489
"stage2_retrieval_rate_at0.75_eval": 0.204254

SyntaxError: invalid syntax (<ipython-input-8-db9367757001>, line 7)

In [ ]:
"num_crops": 9.70676
"stage1_map_at0.25_eval": 0.797403
"stage1_map_at0.50_eval": 0.412992
"stage1_map_at0.75_eval": 0.0683188
"stage1_retrieval_rate_at0.25_eval": 0.997068
"stage1_retrieval_rate_at0.50_eval": 0.992909
"stage1_retrieval_rate_at0.75_eval": 0.976534
"stage2_map_at0.25_eval": 0.614529
"stage2_map_at0.50_eval": 0.401338
"stage2_map_at0.75_eval": 0.0573308
"stage2_retrieval_rate_at0.25_eval": 0.728451
"stage2_retrieval_rate_at0.50_eval": 0.681081
"stage2_retrieval_rate_at0.75_eval": 0.495725